<a href="https://colab.research.google.com/github/whitechocobread/Ai-project/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyautogui


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
import re
import json

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
DATA_PATH = './DATA/' # 데이터 경로 설정
train_data = pd.read_csv(DATA_PATH+'ratings_train.txt', header = 0, delimiter='\t', quoting=3)

train_data['document'][:5]


0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [ ]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)

  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [ ]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_review = []

for review in train_data['document']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_review.append(preprocessing(review,okt,remove_stopwords=True,stop_words= stop_words))
  else:
    clean_train_review.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_review[:4]

[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']]

In [ ]:
#테스트 리뷰도 동일하게 전처리
test_data = pd.read_csv(DATA_PATH + 'ratings_test.txt', header = 0, delimiter='\t', quoting=3)

clean_test_review = []
for review in test_data['document']:
  if type(review) == str:
    clean_test_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_review.append([])

In [ ]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 8 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
train_labels = np.array(train_data['label'])

#평가 데이터
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
#평가 데이터 라벨 벡터화
test_labels = np.array(test_data['label'])

In [ ]:
DEFAULT_PATH  = './content/sample_data/' # 경로지정
DATA_PATH = 'CLEAN_DATA/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)
#전처리 테스트데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)

In [ ]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json


#전처리 데이터 불러오기
DATA_PATH = './content/sample_data/CLEAN_DATA/'
DATA_OUT = './content/sample_data/DATA_OUT/'
INPUT_TRAIN_DATA = 'nsmc_train_input.npy'
LABEL_TRAIN_DATA = 'nsmc_train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA,'rb'))
train_input = pad_sequences(train_input,maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA,'rb'))
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))

In [ ]:
! pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/59/e2/6b155713e8da8274367b41315b66260f9d3c52d9f4b26336ddc3986ba612/matplotlib-3.8.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/ca/2a/d197a412ec474391ee878b1218cf2fe9c6e963903755887fc5654c06636a/contourpy-1.2.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/a0/90/e58e06130ffaf7859959a128fe11ad6533d3e5f4ecf65ecba7464981afdd/fonttools-4.44.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/156.8 kB ? eta -:--:--
     ------- ----------------------------- 30.7/156.8 kB 660.6 kB/s e


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'],'embbeding_size':128, 'num_filters':100,'dropout_rate':0.5, 'hidden_dimension':250,'output_dimension':1}

In [ ]:
class CNNClassifier(tf.keras.Model):

	def __init__(self, **kargs):
		super(CNNClassifier, self).__init__(name=kargs['model_name'])
		self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embbeding_size'])
		self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid',activation = tf.keras.activations.relu,
																		kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3,4,5]]
		self.pooling = layers.GlobalMaxPooling1D()
		self.dropout = layers.Dropout(kargs['dropout_rate'])
		self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
														activation = tf.keras.activations.relu,
														kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
		self.fc2 = layers.Dense(units=kargs['output_dimension'],
														activation=tf.keras.activations.sigmoid,
														kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3.))


	def call(self,x):
		x = self.embedding(x)
		x = self.dropout(x)
		x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis = 1)
		x = self.fc1(x)
		x = self.fc2(x)
		return x

In [ ]:
from tensorflow.keras.models import save_model

model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')])

#검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)
checkpoint_path = DATA_OUT + model_name +'\weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])
# 모델 저장하기
save_model(model,'/model')

./content/sample_data/DATA_OUT/cnn_classifier_kr -- Folder already exists 

Epoch 1/10
264/264 [==============================] - ETA: 0s - loss: 0.4594 - accuracy: 0.7749
Epoch 1: val_accuracy improved from -inf to 0.82173, saving model to ./content/sample_data/DATA_OUT/cnn_classifier_kr\weights.h5
264/264 [==============================] - 16s 55ms/step - loss: 0.4594 - accuracy: 0.7749 - val_loss: 0.3901 - val_accuracy: 0.8217
Epoch 2/10
263/264 [============================>.] - ETA: 0s - loss: 0.3529 - accuracy: 0.8446
Epoch 2: val_accuracy improved from 0.82173 to 0.82960, saving model to ./content/sample_data/DATA_OUT/cnn_classifier_kr\weights.h5
264/264 [==============================] - 14s 52ms/step - loss: 0.3529 - accuracy: 0.8446 - val_loss: 0.3811 - val_accuracy: 0.8296
Epoch 3/10
263/264 [============================>.] - ETA: 0s - loss: 0.2993 - accuracy: 0.8729
Epoch 3: val_accuracy did not improve from 0.82960
264/264 [==============================] - 14s 53ms/step -

PermissionDeniedError: {{function_node __wrapped__MergeV2Checkpoints_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to create a directory: /; Permission denied [Op:MergeV2Checkpoints]

In [ ]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'weights.h5'

test_input = np.load(open(DATA_PATH+INPUT_TEST_DATA,'rb'))
test_input = pad_sequences(test_input,maxlen=test_input.shape[1])
test_label_data = np.load(open(DATA_PATH + LABEL_TEST_DATA, 'rb'))

model.load_weights('content\sample_data\DATA_OUT\cnn_classifier_kr\weights.h5')
model.evaluate(test_input, test_label_data)

1563/1563 [==============================] - 3s 2ms/step - loss: 0.3844 - accuracy: 0.8269


[0.38441741466522217, 0.8269400000572205]

In [ ]:
from tkinter import *
import pyautogui
import pyperclip
import time
import os
import pretext
def InputText(Text):
    pyperclip.copy(Text)
    pyautogui.hotkey("ctrl", "v")

def EnterKey():
    pyautogui.press("Enter")




ChatName = input('채팅방 이름 입력 : ')

kakao_path = 'C:/Users/User/KakaoTalk/KakaoTalk.exe'

os.system(kakao_path)
time.sleep(3)

if os.path.exists(f"ChatTextDataStorage/{ChatName}ChatTextData.txt"):
  pass
else:
  ChatImg = pyautogui.locateOnScreen('./ButtonImages/Chat.png')

  pyautogui.click(ChatImg)
  time.sleep(0.1)

  pyautogui.hotkey("ctrl", "f")
  InputText(ChatName)

  # 방법1 좌표클릭
  """pyautogui.moveRel(150,30)
  time.sleep(0.5)
  pyautogui.doubleClick()"""

  #방법2 걍 엔터버튼
  time.sleep(0.5)
  EnterKey()



  pyautogui.hotkey("ctrl", "s")
  time.sleep(1)

  InputText(f"{ChatName}ChatTextData.txt")
  time.sleep(5)

  EnterKey()
  EnterKey()


  TextData = f"ChatTextDataStorage/{ChatName}ChatTextData.txt"


  # Example usage:
  output_file = "ChatTextDataStorage/sen.txt"
  pretext.process_text_data(TextData, output_file)

ReadTextData =[]

import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
okt = Okt()
tokenizer  = Tokenizer()

prepro_configs = json.load(open('content\sample_data\CLEAN_DATA\data_configs.json','r'))
prepro_configs['vocab'] = word_vocab

tokenizer.fit_on_texts(word_vocab)

MAX_LENGTH = 8 #문장최대길이

with open(output_file, "r", encoding='UTF8') as f:
        ReadTextData = f.read().split("\n")

for i, sentence in enumerate(ReadTextData):
  if i % 2 == 0:
     user = ReadTextData[i]
     pass
  else :
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
    sentence = okt.morphs(sentence, stem=True) # 토큰화
    sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
    vector  = tokenizer.texts_to_sequences(sentence)
    pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩

    model.load_weights('content\sample_data\DATA_OUT\cnn_classifier_kr\weights.h5') #모델 불러오기
    predictions = model.predict(pad_new)
    predictions = float(predictions[0])  # 첫 번째 요소를 스칼라로 변환

    print(f"{user} :" "{:.2f}% 확률로 긍정 리뷰입니다.\n".format(predictions * 100))
    print(sentence)

['정수', '아이기차나 httpsdiscordggPHQYwEgn 으 디코에다가 아니면 드라이브에 가람 으 일단 크롤링 나 유튜브 보고 해보겠음 어떻게하는지 어떻게할건지 엉 코드 써놓게 으 하는사람 올라온나 윤서는 안하고 가람이도 집가고 너도 집가서 하자 집에서 디코로 하자는 의견이 6시 1020 난 이쯤 밥 다막을듯 석식 일단 그 디코에 코드좀 조 이따 할거 파이참이나 비주얼스튜디오에서 핍 안되는거 해결법 찾음 정윤서 너도 나중에 가능 물냉먹어야지 어맞다 기 코드 시작하겟슴 동영상 아 메세지 보내는걸 주석처리안해서 자꾸보내저다 사진 계속 짼다 얘 쥐피티가 낫다 하씨 사진 오호 1장만 하는 이유가 있었군 아니지 첫번째거 해야겟네 저거는 몸풀기 저거 조금만 봐라 애들 시키는거도 그거만시키고 사진 사진 이건 뭘까 엄 사진 사진 사진 httpsdrivegooglecomdrivefolders1cRNQr2tBOhrxv5OFr6vwrXImod59vcDJ ', '나윤', '7시 이후에 만나영 그렇네 없다 음 코드는 좀 부탁혀 자료조사 열심히 해보겠음 헤이 다들 과제하고 있어 가람이는 뭐하니 디코 오자 그럼 나도 할 수 있겠다 걍 시간되는 애들 틈틈이 와서 하자 나도 밥 먹고 갈께 나 는 아 무 것도 먹지 않을려고 해 서경아 나랑 나눠먹으실 일단 메뉴보고 난 닭꼬치 하나 먹어야쥐 간단하다 아휴 신나 9일은 아무도 안 되는뎁 나빼고 가람이 있구나 나 궁금한거 있음 13일에 야자가 없는걸로 알고 있거든 그러면 그 때 특강 남아서 하는건가 야자없으니까 학교마치고 바로 시작하는건가 좀 있다가 보고서 쓰면 봐주십쇼 근데 일단 서론이랑 이론적 배경만 쓰면 되겠지 힘내용 늦었는 시간에 죄송하지만 보고서 서론 쓴거 봐주숍 사진 ', '정윤서', '근데 디스코드 방있음 근데 디코에 1명 안들어왔음 아 이모티콘 다 해볼라고 했는데 좀 아닌데 그래도 해보긴 해봐야 하지 않겠나 ', '정서경', '걍 알아서 하고 넣어두는거 맞나 그거 가 초대는 했는데 디코에서 쥬앙 코드도 짜가야되나 자료찾는거아

C:\Users\User\AppData\Local\Temp\ipykernel_11484\2797500077.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions = float(predictions[0])  # 첫 번째 요소를 스칼라로 변환


5/5 [==============================] - 0s 4ms/step
나윤 :52.02% 확률로 긍정 리뷰입니다.

1/1 [==============================] - 0s 37ms/step
정윤서 :82.95% 확률로 긍정 리뷰입니다.

4/4 [==============================] - 0s 4ms/step
정서경 :36.80% 확률로 긍정 리뷰입니다.

2/2 [==============================] - 0s 5ms/step
강가람 :18.44% 확률로 긍정 리뷰입니다.



In [ ]:
for i, sentence in enumerate(ReadTextData):
  if i % 2 == 0:
     user = ReadTextData[i]
     pass
  else :
    sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
    sentence = okt.morphs(sentence, stem=True) # 토큰화
    sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
    vector  = tokenizer.texts_to_sequences(sentence)
    pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩

    model.load_weights('content\sample_data\DATA_OUT\cnn_classifier_kr\weights.h5') #모델 불러오기
    predictions = model.predict(pad_new)
    predictions = predictions[0].item()  # 배열의 값을 스칼라로 변환
  # 첫 번째 요소를 스칼라로 변환


    print(f"{user} :" "{:.2f}% 확률로 긍정 리뷰입니다.\n".format(predictions * 100))
    print(sentence)

5/5 [==============================] - 0s 3ms/step
정수 :58.43% 확률로 긍정 리뷰입니다.

['아이', '기차', '나', 'ss', '으', '디코', '에다가', '아니다', '드라이브', '에', '가람', '으', '일단', '크롤', '링', '나', '유튜브', '보고', '해보다', '어떻다', '하다', '어떻다', '하다', '건지다', '엉', '코드', '써다', '으', '하다', '사람', '올라오다', '윤서', '안', '하고', '가람이', '도', '지다', '너', '도', '지다', '하자', '집', '에서', '디', '코로', '하자', '의견', '시', '난', '이쯤', '밥', '다', '막다', '석식', '일단', '그', '디코', '에', '코드', '좀', '조', '이따', '하다', '파이', '차다', '비주얼스튜디오', '에서', '피다', '안되다', '해결', '법', '찾다', '정윤서', '너', '도', '나중', '에', '가능', '물냉', '먹다', '어맞다', '기', '코드', '시작', '겟슴', '동영상', '메세지', '보내다', '석', '처리', '안해', '서', '자꾸', '보내다', '저', '다', '사진', '계속', '째다', '얘', '쥐피티', '낫다', '하씨', '사진', '오호', '장만', '하다', '이유', '있다', '아니다', '첫', '번째', '거', '하다', '저', '거', '몸', '풀기', '저', '거', '조금', '만', '보다', '애', '시키다', '그거', '만', '시키다', '사진', '사진', '이건', '뭘', '끄다', '엄', '사진', '사진', '사진', 'ss']
5/5 [==============================] - 0s 3ms/step
나윤 :52.02% 확률로 긍정 리뷰입니다.

['시', '이후', '에', '만', '나영', '그렇다'

In [ ]:
for k, v in dict.items():
    p = 0
    for s in v:
        p += pre(v)
        if p > 0.5:
            print(f"{k} : {p * 100} 확률로 긍정")
        else :
            print(f"{k} : {(1 - p) * 100} 확률로 부정")